# 📌 How to use this notebook (please read once)

Before you start any exercise:

1. Go to the top menu and click Run → Run all

![colab_top_bar_run](https://raw.githubusercontent.com/Haross/DB_pics_nt/main/colab_top_bar_run_all.png)

2. Wait about **1 minute** while everything loads

3. Once everything finishes loading, scroll down to start working with the notebook.

> 👉 **You need to do this once per session (each time you open the notebook).**





## ✍️ Writing and running SQL queries

In this notebook you will see a SQL editor like the one below:

![SQL_editor_run_button](https://raw.githubusercontent.com/Haross/DB_pics_nt/main/colab_sql_gui.png)


Type your SQL queries inside the provided GUI

> Toolbar buttons (in the SQL editor)
>
> * ▶ Blue Run query: runs your SQL and shows results.
>
> * ⟲ Revert to last saved: restores the last saved query for this exercise.
>
> * 🧹 Clear results: clears the displayed output (does not delete data).
>
> * ⌫ Clear editor: clears the query text (does not delete data or history).


Do not edit regular code cells unless explicitly asked

### About the validator

If the validator says your query is incorrect but you believe it is correct:

* Don’t panic 🙂

* The validator is a guideline, not a judge

* In rare cases, a correct query may not be recognized


## 🧠 About the SQL environment setup (do not edit)

You will see a section called “SQL Environment Setup (do not edit)”.

You do not need to understand or modify this

This code is required for the notebook to work

It runs automatically when you click **Run all**

> 👉 **Ignore the section SQL environment setup completely.**

## 🔄 If the SQL editor does not let you type

Sometimes the SQL editor may not respond.

If that happens:

1. Click the ▶ Play button in that same cell that appears at the top-left of the cell. As in the below picture

![SQL_editor_run_button](https://raw.githubusercontent.com/Haross/DB_pics_nt/main/colab_sql_run_Cell_button.png)

2. The editor will refresh and work normally




## 📂 Viewing your previous queries (optional)

After you submit your first query, new files will appear on the left sidebar:

* sql_query_log.csv → latest query per exercise

* sql_query_log_history.csv → all queries tried for each exercise

> ⚠️ These files do not appear until you run at least one query.

You do not need to open these files unless you are curious.

## 🚫 About hidden code

Some cells hide code and show a “Show code” button.

* Do not open these

* If you open one by accident:

  1. Click the cell

  2. Go to View → Show/Hide Code to hide it again

![SQL_editor_run_button](https://raw.githubusercontent.com/Haross/DB_pics_nt/main/colab_view_show_hide_cells.png)


# **SQL Environment Setup (do not edit)**

In [89]:
# @title
%%capture
!pip -q install ipython-sql
!pip -q install ipywidgets
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
DB_FILE = "class.db"
%sql sqlite:///class.db

In [90]:
# @title
%%capture
import os, sqlite3

conn = sqlite3.connect(DB_FILE)
cur = conn.cursor()

#conn.close()

In [91]:
# @title
import hashlib
import pandas as pd

def _dedupe_columns(cols):
    seen = {}
    out = []
    for c in cols:
        k = str(c)
        if k not in seen:
            seen[k] = 0
            out.append(k)
        else:
            seen[k] += 1
            out.append(f"{k}__{seen[k]}")
    return out


def df_fingerprint(
    df: pd.DataFrame,
    *,
    sort_rows=True,
    sort_cols=False,
    normalize_whitespace=True,
    na_token="<NA>",
) -> tuple[str, dict]:
    """
    Returns (sha256_hex, meta) for df, canonicalized.
    - sort_rows: ignores row order (recommended)
    - sort_cols: ignores column order (optional)
    """
    x = df.copy()

    # Make column labels unique (only for hashing purposes)
    x.columns = _dedupe_columns(x.columns)

    # Optional column sorting (usually you want to enforce column order, so keep False)
    if sort_cols:
        x = x.reindex(sorted(x.columns), axis=1)

    # Normalize cell values to strings consistently
    def norm(v):
        if pd.isna(v):
            return na_token
        s = str(v)
        if normalize_whitespace:
            s = " ".join(s.split())
        return s

    x = x.map(norm)

    # Sort rows by all columns to remove ordering differences
    if sort_rows and len(x.columns) > 0 and len(x) > 0:
        x = x.sort_values(by=list(x.columns), kind="mergesort").reset_index(drop=True)
    else:
        x = x.reset_index(drop=True)

    payload = x.to_csv(index=False)  # stable serialization
    h = hashlib.sha256(payload.encode("utf-8")).hexdigest()

    meta = {"rows": int(len(df)), "cols": list(df.columns)}
    return h, meta


def make_df_validator_nospoilers(
    expected_hash: str,
    *,
    required_cols=None,
    exact_cols=False, #If True, the result must contain ONLY the required columns (no missing, no extra). Column order is ignored if sort_cols=True.
    expected_rows=None,
    sort_rows=True, # If True, rows are sorted before computing the fingerprint. This allows answers to be correct even if ORDER BY is missing.
    sort_cols=False, #If True, columns are sorted alphabetically before fingerprinting. This allows SELECT columns in any order to be accepted.
    hide_missing_cols=True,
    hide_row_count=False,
):
    required_cols = required_cols or []

    def validator(df: pd.DataFrame):
        problems = []
        structural_issue = False

        # --- columns ---
        if required_cols:
            missing = [c for c in required_cols if c not in df.columns]
            if missing:
                structural_issue = True
                if hide_missing_cols:
                    problems.append("Wrong number of columns! Make corrections and try again.")
                else:
                    problems.append(f"Missing column(s): {', '.join(missing)}")
                return False, problems

        if exact_cols and required_cols:
            if set(df.columns) != set(required_cols): # same columns order doesn't matter
                structural_issue = True
                return False, ["Wrong number of columns! Make corrections and try again."]

        # If structural issue, stop here (don’t add extra hints like hash mismatch)
        if structural_issue:
            return False, problems

        # --- rows ---
        if expected_rows is not None and len(df) != expected_rows:
            if hide_row_count:
                return False, ["Wrong number of rows! Make corrections and try again."]
            return False, [f"Wrong number of rows! Make corrections and try again."]  # generic but still useful

        # --- values ---
        got_hash, _ = df_fingerprint(df, sort_rows=sort_rows, sort_cols=sort_cols)
        if got_hash != expected_hash:
            return False, ["The result is not correct yet. Make corrections and try again."]

        return True, ["Nice — your output matches the expected result ✅"]

    return validator


import random


SUCCESS_MESSAGES = [
    "👏 Nice!",
    "👏 Great job",
    "👏 Good job",
    "👏 Keep up the good work!",
    "👏 I think you’re getting the hang of this!",
    "👏 Well played",
    "👏 Fantastic! Let’s keep it going",
    "👏 Nicely done",
]


In [92]:
# @title
%%sql
-- Enable foreign key enforcement in SQLite
PRAGMA foreign_keys = ON;

CREATE TABLE director (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    birth_year INTEGER
);

CREATE TABLE movie (
    id INTEGER PRIMARY KEY,
    title TEXT NOT NULL,
    production_year INTEGER,
    director_id INTEGER,
    FOREIGN KEY (director_id) REFERENCES director(id)
);

INSERT INTO director (id, name, birth_year) VALUES
(1, 'Alfred Hitchcock', 1899),
(2, 'Steven Spielberg', 1946),
(3, 'Woody Allen', 1935),
(4, 'Quentin Tarantino', 1963),
(5, 'Pedro Almodóvar', 1949);


INSERT INTO movie (id, title, production_year, director_id) VALUES
(1, 'Psycho', 1960, 1),
(2, 'Saving Private Ryan', 1998, 2),
(3, 'Schindler''s List', 1993, 2),
(4, 'Midnight in Paris', 2011, 3),
(5, 'Sweet and Lowdown', 1993, 3),
(6, 'Pulp Fiction', 1994, 4),
(7, 'Talk to Her', 2002, 5),
(8, 'The Skin I Live In', 2011, 5);



 * sqlite:///class.db
(sqlite3.OperationalError) table director already exists
[SQL: CREATE TABLE director (
    id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    birth_year INTEGER
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [93]:
# @title
import sqlite3
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from pathlib import Path
import csv
from datetime import datetime
import html as _html



def make_sql_runner(
    conn,
    runner_id: str,
    default_sql=None,
    select_only=True,
    validator=None,
    dedupe=True,
    description_md=None,
    hint_enabled=False,
    hint_md=None,
    schema_tables=None,
):
    # ---------- helpers ----------
    def md_to_html(md: str) -> str:
        try:
            import markdown as _md
            return _md.markdown(md)
        except Exception:
            return "<br>".join(_html.escape(md).splitlines())

    # ---------- persistence ----------
    LOG_ALL_FILE = Path("sql_query_log.csv")
    LOG_LATEST_FILE = Path("sql_query_latest.csv")

    def _append_history(runner_id: str, sql: str, log_path: Path = LOG_ALL_FILE):
        is_new = not log_path.exists()
        with log_path.open("a", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            if is_new:
                w.writerow(["ts", "runner_id", "sql"])
            w.writerow([datetime.now().isoformat(timespec="seconds"), runner_id, sql])

    def _load_latest_map(latest_path: Path = LOG_LATEST_FILE) -> dict:
        if not latest_path.exists():
            return {}
        latest = {}
        with latest_path.open("r", newline="", encoding="utf-8") as f:
            r = csv.DictReader(f)
            for row in r:
                latest[row["runner_id"]] = row["sql"]
        return latest

    def _save_latest_map(latest: dict, latest_path: Path = LOG_LATEST_FILE):
        with latest_path.open("w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(["runner_id", "sql"])
            for rid, sql in latest.items():
                w.writerow([rid, sql])

    latest_map = _load_latest_map()
    last_saved = latest_map.get(runner_id)
    if last_saved is not None:
        initial_sql = last_saved
    else:
        initial_sql = default_sql or ""
    # ---------- UI chrome (CSS) ----------
    display(HTML("""
      <style>
        /* =========================
          Global runner bounds
          ========================= */
        .sql-runner{
          max-width: 100% !important;
          box-sizing: border-box !important;
          padding-right: 18px;   /* keep resize handle away from notebook scrollbar */
          padding-bottom: 12px;
          overflow-x: hidden;
        }

        .sql-runner .widget-box,
        .sql-runner .widget-vbox,
        .sql-runner .widget-hbox{
          width: 100% !important;
          max-width: 100% !important;
          box-sizing: border-box !important;
        }

        /* =========================
          Description / Hint boxes
          ========================= */
        .sql-desc{
          border-left: 4px solid #1a73e8;
          background: #f5f9ff;
          padding: 10px 12px;
          margin: 6px 0 10px 0;
          border-radius: 6px;
          font-size: 14px;
          line-height: 1.5;
        }
        .sql-hintbox{
          border-left: 4px solid #fbbc04;
          background: #fff8e1;
          padding: 10px 12px;
          margin: 8px 0 10px 0;
          border-radius: 6px;
          font-size: 14px;
          line-height: 1.5;
        }

        /* =========================
          Editor + toolbar panel
          ========================= */
        .sql-runner .sql-panel{
          border: 1px solid #d0d7de;
          border-radius: 12px;
          background: #f6f8fa;
          overflow: hidden;
        }

        /* Textarea wrapper adapts to resized textarea */
        .sql-runner .widget-textarea{
          height: auto !important;
        }

        /* Base textarea behavior (resizable) */
        .sql-runner .widget-textarea textarea{
          height: 95px;                 /* initial size */
          min-height: 120px !important;
          resize: vertical !important;
          width: 100% !important;
          max-width: 100% !important;
          box-sizing: border-box !important;
        }

        /* Editor look */
        .sql-runner .sql-editor textarea{
          background: #ffffff;
          border: 0 !important;          /* panel provides border */
          border-radius: 0 !important;
          padding: 12px !important;
          font-family: ui-monospace, SFMono-Regular, Menlo, Monaco, Consolas, "Liberation Mono", "Courier New", monospace;
          font-size: 13px;
          line-height: 1.4;
        }

        /* Toolbar */
        .sql-runner .sql-toolbar{
          border-top: 1px solid #d0d7de;
          background: #f6f8fa;
          margin: 0 !important;
          padding: 6px 10px !important;
        }

        .sql-runner .sql-toolbar .widget-button{
          min-width: 40px !important;
          width: 40px !important;
          height: 40px !important;
          border-radius: 8px !important;
          font-size: 18px !important;

          background: transparent !important;
          border: 1px solid transparent !important;
          box-shadow: none !important;
          color: #24292f !important;
        }
        .sql-runner .sql-toolbar .widget-button:hover{
          background: #eaeef2 !important;
          border-color: #d0d7de !important;
        }
        .sql-runner .sql-toolbar .widget-button.mod-primary{
          background: #1a73e8 !important;
          border-color: #1a73e8 !important;
          color: #ffffff !important;
        }
        .sql-runner .sql-toolbar .widget-button.mod-primary:hover{
          filter: brightness(0.95);
        }
        .sql-runner .sql-toolbar .hint{
          color: #57606a;
          font-size: 12px;
          margin-left: 12px;
        }

        /* =========================
          Tabs results panel (dock)
          ========================= */

        .sql-runner .sql-tabs-panel{
          border: 1px solid #d0d7de;
          border-radius: 10px;
          background: #ffffff;
          overflow: hidden;
        }

        /* Remove default focus rings */
        .sql-runner .widget-tab:focus,
        .sql-runner .widget-tab :focus{
          outline: none !important;
          box-shadow: none !important;
        }

        /* Let outer panel own the border */
        .sql-runner .sql-tabs-panel .widget-tab{
          border: 0 !important;
          background: transparent !important;
        }

        /* Tab bar */
        .sql-runner .widget-tab > .p-TabBar{
          background: #f6f8fa !important;
          border-bottom: 1px solid #d0d7de !important;
          padding: 0 6px !important;
        }

        /* Tabs */
        .sql-runner .p-TabBar-tab{
          margin: 0 6px 0 0 !important;
          padding: 6px 12px !important;
          font-size: 13px !important;
          line-height: 18px !important;
          color: #57606a !important;

          background: transparent !important;
          border: 1px solid transparent !important;
          border-bottom: 0 !important;
          border-radius: 8px 8px 0 0 !important;

          position: relative; /* for ::after indicator */
        }

        .sql-runner .p-TabBar-tab:hover{
          background: #eaeef2 !important;
          border-color: #d0d7de !important;
        }

        /* Active tab */
        .sql-runner .p-TabBar-tab.p-mod-current{
          background: #ffffff !important;
          color: #24292f !important;
          border-color: #d0d7de !important;
          border-bottom: 1px solid #ffffff !important; /* merges into content */
          font-weight: 500 !important;
          z-index: 2;
        }

        /* Blue indicator INSIDE the tab (prevents "blue line below") */
        .sql-runner .p-TabBar-tab.p-mod-current::after{
          content: "";
          position: absolute;
          left: 12px;
          right: 12px;
          bottom: 4px;      /* <-- inside the tab; NOT -1px */
          height: 2px;
          background: #1a73e8;
          border-radius: 2px;
        }

        /* Tab content */
        .sql-runner .widget-tab > .p-TabPanel{
          padding: 10px !important;
          background: #ffffff !important;
        }


        /* --- HARD OVERRIDES: remove Lumino/Colab active-tab blue indicator --- */
        .sql-runner .p-TabBar-tab.p-mod-current{
          box-shadow: none !important;      /* Lumino often draws the blue line here */
          background-image: none !important;
        }

        /* Some themes use ::before as the underline */
        .sql-runner .p-TabBar-tab.p-mod-current::before{
          content: none !important;
          display: none !important;
        }

        /* Some themes apply focus-visible outline/underline */
        .sql-runner .p-TabBar-tab:focus-visible,
        .sql-runner .p-TabBar-tab.p-mod-current:focus-visible{
          outline: none !important;
          box-shadow: none !important;
        }



        /* =========================
          Remove Lumino inner divider inside tab contents
          ========================= */
        .sql-runner .sql-tabs-panel .widget-tab-contents,
        .sql-runner .sql-tabs-panel .p-TabPanel-tabContents{
          border: 0 !important;
          box-shadow: none !important;
          outline: none !important;
          background: transparent !important;
        }





        /* =========================
          Resizable tabs container (stable)
          ========================= */

        /* Make the OUTER bordered panel resizable */
        .sql-runner .sql-tabs-panel{
          resize: vertical;
          overflow: hidden;          /* important: prevents the tab bar from getting clipped/overlapped */
          min-height: 220px;         /* prevents collapsing into the tabs */
        }

        /* Force the Tab widget to use a vertical flex layout */
        .sql-runner .sql-tabs-panel .widget-tab{
          height: 100% !important;
          display: flex !important;
          flex-direction: column !important;
        }

        /* Tab bar stays fixed at top */
        .sql-runner .sql-tabs-panel .widget-tab > .p-TabBar{
          flex: 0 0 auto !important;
        }

        /* Tab content area becomes the flexible, scrollable region */
        .sql-runner .sql-tabs-panel .widget-tab > .p-TabPanel{
          flex: 1 1 auto !important;
          overflow: auto !important;
          min-height: 140px;         /* keeps content area usable even when resized smaller */
          box-sizing: border-box !important;
        }





        /* =========================
          Validation box
          ========================= */
        .sql-validation{
          position: relative;
          padding: 10px 38px 10px 12px; /* extra right padding for ✕ */
          margin: 8px 0 10px 0;
          border-radius: 6px;
          font-size: 14px;
          line-height: 1.5;
        }
        .sql-validation.ok{
          border-left: 4px solid #2e7d32;
          background: #e8f5e9;
        }
        .sql-validation.err{
          border-left: 4px solid #b00020;
          background: #ffebee;
        }
        .sql-validation .close{
          position: absolute;
          top: 8px;
          right: 10px;
          cursor: pointer;
          user-select: none;
          opacity: 0.65;
          font-weight: 700;
        }
        .sql-validation .close:hover{
          opacity: 1;
        }
        .sql-validation ul{
          margin: 6px 0 0 18px;
        }



      </style>
      """))


    # ---------- widgets ----------

    desc_widget = None
    if description_md:
        desc_widget = widgets.HTML(
            value=f"<div class='sql-desc'>{md_to_html(description_md)}</div>"
        )

    # Hint components (created only if enabled + text provided)
    show_hint_ui = bool(hint_enabled and hint_md)

    hint_btn = None
    hint_box = None
    if show_hint_ui:
        hint_btn = widgets.Button(
            description="💡",
            tooltip="Show/hide hint",
            layout=widgets.Layout(width="40px", height="40px"),
        )
        hint_visible = False

        hint_html = widgets.HTML(value=f"<div class='sql-hintbox'>{md_to_html(hint_md)}</div>")
        hint_box = widgets.Box([hint_html], layout=widgets.Layout(display="none"))

        def on_hint_click(_):
            nonlocal hint_visible
            hint_visible = not hint_visible
            hint_box.layout.display = "block" if hint_visible else "none"

        hint_btn.on_click(on_hint_click)


    # ---------- validation banner ----------

    validation_widget = widgets.HTML(value="")
    validation_nonce = 0

    def hide_validation():
        validation_widget.value = ""


    def show_validation(ok: bool, problems_or_msg):
        nonlocal validation_nonce
        validation_nonce += 1

        if isinstance(problems_or_msg, str):
            problems = [problems_or_msg] if problems_or_msg else []
        else:
            problems = list(problems_or_msg or [])

        cls = "ok" if ok else "err"

        if ok:
            title = random.choice(SUCCESS_MESSAGES)
            message = ""   # no extra text needed
        else:
            title = "🙁 Not correct yet"
            message = " ".join(problems)

        box_id = f"val_{runner_id}_{validation_nonce}"

        validation_widget.value = f"""
          <div id="{box_id}" class="sql-validation {cls}">
            <div class="close" onclick="document.getElementById('{box_id}').remove()">✕</div>
            <b>{_html.escape(title)}</b>
            { _html.escape(message) }
          </div>
        """


    # ---------- end validation banner ----------

    box = widgets.Textarea(
        value=initial_sql,
        placeholder="Type your SQL query here...",
        description="",
        layout=widgets.Layout(width="100%")
    )
    box.add_class("sql-editor")

    results_out = widgets.Output()
    schema_out = widgets.Output()

    results_box = widgets.Box(
        [results_out],
        layout=widgets.Layout(width="100%", padding="8px")
    )
    schema_box = widgets.Box(
        [schema_out],
        layout=widgets.Layout(width="100%",  padding="8px")
    )


    tabs = widgets.Tab(children=[results_box, schema_box], layout=widgets.Layout(width="100%"))
    tabs.set_title(0, "Query results")
    tabs.set_title(1, "Schema Database")

    # Toolbar buttons
    run_btn = widgets.Button(
        description="▶",
        tooltip="Run query",
        layout=widgets.Layout(width="40px", height="40px"),
        button_style="primary"
    )
    revert_btn = widgets.Button(description="⟲", tooltip="Revert to last saved", layout=widgets.Layout(width="40px", height="40px"))

    reset_btn = None
    if default_sql:
        reset_btn = widgets.Button(
            description="↩",
            tooltip="Reset to default SQL",
            layout=widgets.Layout(width="40px", height="40px")
        )

    clear_results_btn = widgets.Button(description="🧹", tooltip="Clear results output", layout=widgets.Layout(width="40px", height="40px"))
    clear_query_btn = widgets.Button(description="⌫", tooltip="Clear query editor", layout=widgets.Layout(width="40px", height="40px"))

    status = widgets.HTML('<span class="hint"></span>')

    def set_status(msg: str):
        status.value = f'<span class="hint">{msg}</span>' if msg else '<span class="hint"></span>'

    # Toolbar composition
    left_items = [run_btn]
    if hint_btn:
        left_items.append(hint_btn)


    left_items.append(revert_btn)
    if reset_btn:
        left_items.append(reset_btn)
    left_items.extend([clear_results_btn, clear_query_btn])

    left = widgets.HBox(left_items, layout=widgets.Layout(gap="8px", align_items="center"))
    right = widgets.HBox([status], layout=widgets.Layout(justify_content="flex-end", align_items="center"))

    toolbar = widgets.HBox(
        [left, right],
        layout=widgets.Layout(width="100%", align_items="center", justify_content="space-between")
    )
    toolbar.add_class("sql-toolbar")

    # ---------- schema renderer ----------
    def render_schema():
      with schema_out:
          clear_output()
          try:
              all_tables = pd.read_sql_query(
                  """
                  SELECT name
                  FROM sqlite_master
                  WHERE type='table' AND name NOT LIKE 'sqlite_%'
                  ORDER BY name;
                  """,
                  conn
              )["name"].tolist()

              if not all_tables:
                  display(HTML("<b>No tables found.</b>"))
                  return

              # Apply filter if provided
              if schema_tables:
                  # keep only tables that actually exist (preserve requested order)
                  tables = [t for t in schema_tables if t in all_tables]
                  missing = [t for t in schema_tables if t not in all_tables]

                  if missing:
                      display(HTML(
                          "<div style='margin:6px 0 10px 0;color:#b00020'>"
                          f"<b>Note:</b> table(s) not found: {', '.join(_html.escape(x) for x in missing)}"
                          "</div>"
                      ))
              else:
                  tables = all_tables

              if not tables:
                  display(HTML("<b>No matching tables to display.</b>"))
                  return

              items = []
              titles = []

              for t in tables:
                  info = pd.read_sql_query(f"PRAGMA table_info('{t}');", conn)
                  info = info[["name", "type", "notnull", "dflt_value", "pk"]]

                  out = widgets.Output()
                  with out:
                      display(info.style.format(na_rep="NULL").hide(axis="index"))

                  items.append(out)
                  titles.append(t)

              acc = widgets.Accordion(children=items)
              for i, t in enumerate(titles):
                  acc.set_title(i, t)

              acc.selected_index = 0 if len(tables) == 1 else None
              display(acc)

          except Exception as e:
              display(HTML(f"<pre style='color:#b00020'>Error:\n{e}</pre>"))

    def on_tab_change(change):
        if change["name"] == "selected_index" and change["new"] == 1:
            render_schema()

    tabs.observe(on_tab_change)

    # ---------- actions ----------
    def run_query(_):
        nonlocal last_saved, latest_map

        q = box.value.strip()
        with results_out:
            clear_output()

            if not q:
                display(HTML("<b>Please type a query.</b>"))
                tabs.selected_index = 0
                set_status("No query to run.")
                return

            norm = lambda s: " ".join(s.split())
            changed = (norm(q) != norm(last_saved or ""))

            if (not dedupe) or changed:
                _append_history(runner_id, q)
                latest_map = _load_latest_map()
                latest_map[runner_id] = q
                _save_latest_map(latest_map)
                last_saved = q

            if select_only and not q.lower().lstrip().startswith(("select", "with")):
                display(HTML("<b>Only SELECT/WITH queries are allowed.</b>"))
                tabs.selected_index = 0
                set_status("Blocked: only SELECT/WITH allowed.")
                return

            # disable toolbar during execution
            for b in (run_btn, revert_btn, clear_results_btn, clear_query_btn):
                b.disabled = True
            if reset_btn:
                reset_btn.disabled = True
            if hint_btn:
                hint_btn.disabled = True

            try:
                if q.lower().lstrip().startswith(("select", "with")):
                    df = pd.read_sql_query(q, conn)
                    display(df.style.format(na_rep="NULL").hide(axis="index"))
                    set_status(f"Returned {len(df)} row(s).")

                    if validator:
                        ok, problems = validator(df)   # (bool, list[str]) or (bool, str)
                        show_validation(ok, problems)
                    else:
                        hide_validation()

                else:
                    cur = conn.cursor()
                    cur.executescript(q)
                    conn.commit()
                    display(HTML("<b>✅ Query executed.</b>"))
                    set_status("Query executed.")

                tabs.selected_index = 0

            except Exception as e:
                display(HTML(f"<pre style='color:#b00020'>Error:\n{e}</pre>"))
                tabs.selected_index = 0
                set_status(f"Error running query.")
            finally:
                for b in (run_btn, revert_btn, clear_results_btn, clear_query_btn):
                    b.disabled = False
                if reset_btn:
                    reset_btn.disabled = False
                if hint_btn:
                    hint_btn.disabled = False

    def revert_query(_):
        nonlocal last_saved
        latest_map_local = _load_latest_map()
        saved = latest_map_local.get(runner_id)
        box.value = saved if saved is not None else (default_sql or "")
        set_status("Reverted to last saved." if saved is not None else "No saved query — reverted.")

    def reset_to_default(_):
        if default_sql:
          box.value = default_sql
          set_status("Reset to default SQL.")

    def clear_results(_):
        results_out.clear_output()
        set_status("Cleared results output.")

    def clear_query(_):
        box.value = ""
        set_status("Cleared query editor.")

    run_btn.on_click(run_query)
    revert_btn.on_click(revert_query)
    if reset_btn:
        reset_btn.on_click(reset_to_default)
    clear_results_btn.on_click(clear_results)
    clear_query_btn.on_click(clear_query)

    # ---------- layout ----------
    elements = []
    if desc_widget:
        elements.append(desc_widget)
    if hint_box:
        elements.append(hint_box)
    elements.append(validation_widget)

    editor_panel = widgets.VBox([box, toolbar])
    editor_panel.add_class("sql-panel")

    spacer = widgets.Box(layout=widgets.Layout(height="10px"))


    #elements.extend([editor_panel, spacer,tabs])
    tabs_panel = widgets.VBox([tabs])
    tabs_panel.add_class("sql-tabs-panel")
    tabs_panel.layout.height = "230px"

    elements.extend([editor_panel, spacer, tabs_panel])

    ui = widgets.VBox(elements, layout=widgets.Layout(width="100%"))

    ui.add_class("sql-runner")
    display(ui)

    render_schema()
    set_status("Ready.")


In [94]:
# @title
#Professor- answer validator generator
solution_sql = """
SELECT * FROM movie
"""
solution_df = pd.read_sql_query(solution_sql, conn)
expected_hash, meta = df_fingerprint(solution_df, sort_rows=True,sort_cols=True)
expected_hash, meta

('d1fffc55a4ac0c354f72e14f31234d353e815b430201efbf36b6b34649578429',
 {'rows': 8, 'cols': ['id', 'title', 'production_year', 'director_id']})

# Working with Multiple Tables

In the previous notebook, you learned how to retrieve data from a single table
and how to filter rows using conditions.

In real-world databases, however, data is rarely stored in just one table.
Instead, information is usually split across **multiple tables** to reduce
redundancy and improve organization.

As a result, it is often necessary to retrieve and combine data from more than
one table at a time.

In this notebook, you will learn how to work with data from multiple tables and
how to join results to answer more complex questions.


## Overview of the Tables

Before working with data from multiple tables, it is important to understand
the structure of the tables used in the examples.

In this notebook, we will work with two tables: `car` and `person`.

The tables have the following columns:

>- person (id, name, age)
>- car (id, brand, model, price, owner_id)

The `car` table stores information about cars, while the `person` table stores
information about their owners. Each car is linked to its owner through the
`owner_id` column in the `car` table, which references the `id` column in the
`person` table.

This relationship allows us to associate each car with the corresponding person.

The relationship between these tables will be used to combine data in the following sections.

## Using Multiple Tables in the FROM Clause

As seen earlier, tables in a relational database are often connected through
shared identifiers.

For example, in our dataset, the `car` table contains an `owner_id` column.
This column references the `id` column in the `person` table, allowing us to
associate each car with its owner.

There are several ways to retrieve data from multiple tables at the same time.
We will begin with the simplest approach.

```sql
SELECT *
FROM person, car;
```

This query lists two tables in the `FROM` clause, separated by a comma.
At this point, the database combines every row from `person` with every row
from `car`.

The result may not be what you expect, so let’s examine it carefully.

In [95]:
# @title Practice 1
validator = make_df_validator_nospoilers(
    expected_hash="f4d077a0a7e220a2d420e8b6d3dca035f25798f4fd1535635ad9b078b95529b0",
    required_cols=['id','title','production_year',
   'director_id',
   'id',
   'name',
   'birth_year'],
    expected_rows=40,
    sort_rows=True,
    sort_cols=True
)
make_sql_runner(
    conn,
    runner_id="ex1",
    description_md="""
### Practice 1
Get all the data from two tables: movie and director.

If there are 8 movies and 5 directors, **how many rows will we get in our result?** Think about it before you click **Run** and check the result.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## Joining Tables Using a Condition

When multiple tables are listed in the `FROM` clause without a condition,
SQL combines every row from one table with every row from the other.
This produces all possible row combinations.

For example, if the `person` table contains 5 rows and the `car` table
contains 8 rows, the result will contain 5 × 8 = 40 rows.

This happens because SQL has no information about how the rows in the two
tables are related.

To restrict the result to meaningful combinations, we must specify a
condition that defines how the tables are connected:

```sql
SELECT *
FROM person, car
WHERE person.id = car.owner_id;
````

This condition tells SQL to return only those row combinations where the
`id` value from the `person` table matches the `owner_id` value from the
`car` table. As a result, each car is associated with its correct owner.

When working with multiple tables, column names should be qualified with
the table name to avoid ambiguity. The table name and column name are
separated by a dot (`.`).

For example:

> * **car**.owner_id
>
> * **person**.id



In [96]:
# @title Practice 2
validator = make_df_validator_nospoilers(
    expected_hash="2ff11bbe86f2eb533db433b87be7c0c74d89b5ac3488330c04d8d47a11d75456",
    required_cols=['id',
   'title',
   'production_year',
   'director_id',
   'id',
   'name',
   'birth_year'],
    expected_rows=8,
    sort_rows=True,
    sort_cols=True
)


make_sql_runner(
    conn,
    runner_id="ex2",
    description_md="""
### Practice 2
Select all columns from tables `movie` and `director` in such a way that a movie is shown together with its director.

For the sake of this exercise, join `movie` and `director` tables in this exact order (`movie, director`). Otherwise the validator would not work
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## The JOIN Keyword

Joining tables is a common operation in SQL.
Because of this, SQL provides a dedicated keyword for it: `JOIN`.

Using `JOIN` allows us to express relationships between tables more clearly
than listing multiple tables in the `FROM` clause with a condition.

> 💡 **Did you know?**  
> SQL supports several types of joins.  
> In this notebook, we will focus on the most basic and commonly used one.

## Joining Tables Using JOIN

The following example shows how to join two tables using the `JOIN` keyword:

```sql
SELECT *
FROM person
JOIN car
  ON person.id = car.owner_id;
```  
In this query, the tables person and car are joined using the `JOIN` keyword.

SQL also needs to know **how** the tables are related. This is specified using
the `ON` keyword, followed by a condition.

The condition `person.id = car.owner_id` tells SQL to join only those rows
where the `id` value from the person table matches the `owner_id` value
from the `car` table.


In [97]:
# @title Practice 3
validator = make_df_validator_nospoilers(
    expected_hash="2ff11bbe86f2eb533db433b87be7c0c74d89b5ac3488330c04d8d47a11d75456",
    required_cols=['id',
   'title',
   'production_year',
   'director_id',
   'id',
   'name',
   'birth_year'],
    expected_rows=8,
    sort_rows=True,
    sort_cols=True
)

make_sql_runner(
    conn,
    runner_id="ex3",
    description_md="""
### Practice 3
Use the new construction **JOIN ... ON** to join rows from the tables **movie** and **director** in such a way that a movie is shown together with its director. Display **all** columns from both tables.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## Displaying Specific Columns

In many cases, we do not need all columns from the joined tables.
Instead, we can select only the columns that are relevant to the query.

For example, the following query returns the model of each car and the
name of its owner:


```sql
SELECT
  person.name,
  car.model
FROM person
JOIN car
  ON person.id = car.owner_id;
```  
Instead of using the asterisk (`*`), the query explicitly lists the columns
to be displayed.

When working with multiple tables, column names should be prefixed with
the table name and separated by a dot (`.`). This makes it clear which
table each column belongs to.

In [98]:
# @title Practice 4
validator = make_df_validator_nospoilers(
    expected_hash="ab2532437374282a9ac7b29e3005ddce2cbad196fc57d4e6cdd3d400f9cf90b5",
    required_cols=['name', 'title'],
    expected_rows=8,
    sort_rows=True,
    sort_cols=True
)

make_sql_runner(
    conn,
    runner_id="ex4",
    description_md="""
### Practice 4
Select **director name** and **movie title** from tables **movie** and **director** in such a way that a movie is shown together with its director.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## Referring to Columns Without Table Names

In the previous examples, column names were prefixed with their table names.
This is good practice, especially when working with multiple tables.

However, prefixing the table name is **only required** when there is a **risk of
ambiguity**. If a column name appears in more than one table, the table name
must be specified. **If the column name is unique across the tables used in
the query, the table name can be omitted**

For example:

```sql
SELECT
  name,
  model
FROM person
JOIN car
  ON person.id = owner_id;
```
  
In this query, the column `name` exists only in the `person` table, and the
column `model` exists only in the `car` table. Because these column names are
unique, the table names can be omitted.

Similarly, the column `owner_id` exists only in the `car` table, so it does
not need to be prefixed with the table name.

On the other hand, both tables contain a column named `id`. For this reason,
the condition must explicitly reference `person.id` to avoid ambiguity.

In [99]:
# @title Practice 5
validator = make_df_validator_nospoilers(
    expected_hash="ab2532437374282a9ac7b29e3005ddce2cbad196fc57d4e6cdd3d400f9cf90b5",
    required_cols=['name', 'title'],
    expected_rows=8,
    sort_rows=True,
    sort_cols=True
)
make_sql_runner(
    conn,
    runner_id="ex5",
    description_md="""
### Practice 5
Select **director name** and **movie title** from the movie and director tables in such a way that a movie is shown together with its director. Don't write table names in the `SELECT` clause.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## Renaming Columns with Aliases

So far, column names have appeared in the result exactly as they are defined in the table.  
Sometimes, however, we want the output to be clearer or more descriptive.

For example, the column named `id` is shown simply as `id` in the result.  
We can change how it appears by giving it a new name.

```sql
SELECT
  person.id AS person_id,
  car.id AS car_id
FROM person
JOIN car
  ON person.id = car.owner_id;
```  
After the column name (for example, `person.id`), we use the keyword AS
followed by the new name we want to display (`person_id`).

This new name is called an **alias**.
An alias is temporary and does not change the actual column name in the database.
It only affects how the column appears in the result of this specific query.

Using aliases is especially useful when multiple tables contain columns
with the same name. By default, the result does not indicate which table
a column comes from.

In this example, both tables have a column called `id`, so we rename them
to `person_id` and `car_id`. This makes the result easier to read and
avoids confusion.



In [100]:
# @title Practice 6
validator = make_df_validator_nospoilers(
    expected_hash="d560152db28ea1dfd1f3ef9917f340a5fc68b915a7be61d42dd9517bdb0d6006",
    required_cols=['movie_title', 'name'],
    expected_rows=8,
    sort_rows=True,
    sort_cols=True
)
make_sql_runner(
    conn,
    default_sql="""SELECT
  title,
  name
FROM movie
JOIN director
  ON director_id = director.id;""",
    runner_id="ex6",
    description_md="""
### Practice 6
In this exercise, show the *title* column as *movie_title*. We wrote the query from the previous exercise in the template, so you just have to add a proper alias.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## Filtering the joined tables

So far, our queries have returned all matching rows from the joined tables.  
Often, we want to limit the result to rows that meet an additional condition.

Filtering joined results is often necessary, and the same rules apply as when
filtering a single table.

```sql
SELECT
  person.id,
  car.model
FROM person
JOIN car
  ON person.id = car.owner_id
WHERE person.age < 25;
```
The new part here is the `WHERE` clause. Now we only look for such connections of cars and their owners where the owner is below 25.

When working with multiple tables, it is important to include the table name
in the condition, such as person.age. This avoids ambiguity and makes it
clear which column the condition refers to.



In [101]:
# @title Practice 7
validator = make_df_validator_nospoilers(
    expected_hash="0a7e76a8c2875dcdf8ab7e517184e6e0923083ac67004956627f4529ca17cad3",
    required_cols=['id',
   'title',
   'production_year',
   'director_id',
   'id',
   'name',
   'birth_year'],
    expected_rows=3,
    sort_rows=True,
    sort_cols=True
)

make_sql_runner(
    conn,
    runner_id="ex7",
    description_md="""
### Practice 7
Select all columns from tables *movie* and *director* in such a way that a movie is shown together with its director. Select only those movies which were made after 2000. In the joining condition, let the first table be *movie* and the second table be *director*.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## Filtering the joined tables continued

Conditions in the `WHERE` clause can also be based on text values.
For example, the following query selects only cars of a specific brand:

```sql
SELECT
  person.id,
  car.model
FROM person
JOIN car
 ON person.id = car.owner_id
WHERE car.brand = 'Fiat';
```
In this query, the result is restricted to rows where the value of the
brand column in the car table is 'Fiat'.



In [102]:
# @title Practice 8
validator = make_df_validator_nospoilers(
    expected_hash="24f9b4cf85fe2822aac458f0e2ad694e0d9913d0f9f24fac0a6082f2153b87e0",
    required_cols=['id',
   'title',
   'production_year',
   'director_id',
   'id',
   'name',
   'birth_year'],
    expected_rows=2,
    sort_rows=True,
    sort_cols=True
)


make_sql_runner(
    conn,
    runner_id="ex8",
    description_md="""
### Practice 8
Select all columns from tables *movie* and *director* in such a way that a movie is shown together with its director. Select only those movies which were directed by Steven Spielberg.
""",
    validator=validator,
    schema_tables=["movie","director"]
    )



## In-class assignment: Put your skills into practice
Let's put into practice everything we've learned so far. Are you ready? This example is going to be slightly more complicated, so make sure you remember everything from this part of the notebook.

In [103]:
# @title In-class exercise 1

make_sql_runner(
    conn,
    runner_id="inclass1",
    description_md="""
### In-class exercise 1
Select the **title** and **production_year** columns from the **movie** table, and the **name** and **birth_year** columns from the **director** table in such a way that a movie is shown together with its director.

Show the column **birth_year** as **born_in**. Select only those movies which were filmed when their director was younger than 40 (i.e. the difference between **production_year** and **birth_year** must be less than 40).
""",
    schema_tables=["movie","director"]
    )



## In-class exercise 2: Further practice
Excellent! We have one more exercise for you before the end of this part. You are going to write your longest SQL query ever and you'll apply knowledge from both of the notebooks you've mastered so far. Are you ready

In [104]:
# @title In-class exercise 2

make_sql_runner(
    conn,
    runner_id="inclass2",
    description_md="""
### In-class exercise 2
Select the **id**, **title**, and **production_year** columns from the **movie** table, and the **name** and **birth_year** columns from the **director** table in such a way that a movie is shown together with its director. Show the column **birth_year** as **born_in** and the column **production_year** as **produced_in**. Select only those movies:

* whose title contains a letter 'a' and which were filmed after 2000,
**or**
* whose director was born between 1945 and 1995.
""",
    schema_tables=["movie","director"]
    )

